## Data Driven Business Decisions for Chinook Record Store 

The following project addresses four business decisions for a fictional Chinook Records management based on their available data. These decisions to be made are:

* Which newcomer artist/s should be advertised for US market?
* Whether there are sales representatives performing better or worse than others
* Which countries have potential to grow?
* Whether the company should stop buying albums as a whole and start buying only popular tracks from record labels to reduce costs

The dataset to be consulted is provided at: https://github.com/lerocha/chinook-database


Let us first write down the helper functions we will use throughout the project:

In [1]:
import sqlite3 as sq
import pandas as pd

from IPython.display import display


#Returns a DF from a SQL query or table
def run_query(query_or_table,db_name = 'chinook.db', index_col = None ):                   
    with sq.connect('chinook.db') as con:
        df = pd.read_sql(query_or_table,con,index_col = index_col)
        return df

#Runs SQL queries but doesn't return tables (like CREATE VIEW)
def run_command(query_str,db_name = 'chinook.db'):         
    with sq.connect('chinook.db') as con:                  
        con.isolation_level = None                         #autocommit any changes
        con.execute(query_str)

#Get the structure of the database        
def show_sqlmaster(check_for="", db_name = 'chinook.db'):    
    if (check_for): 
        query = "select name from sqlite_master where type in "+check_for+";"
    else:
        query = "select * from sqlite_master;"

    return run_query(query,db_name)


Before we start any analysis, let us explore the structure of the database: 

In [2]:
show_sqlmaster()

,type,name,tbl_name,rootpage,sql
0,table,album,album,2,CREATE TABLE [album]\n(\n [album_id] INTEGE...
1,table,artist,artist,3,CREATE TABLE [artist]\n(\n [artist_id] INTE...
2,table,customer,customer,4,CREATE TABLE [customer]\n(\n [customer_id] ...
3,table,employee,employee,5,CREATE TABLE [employee]\n(\n [employee_id] ...
4,table,genre,genre,6,CREATE TABLE [genre]\n(\n [genre_id] INTEGE...
5,table,invoice,invoice,7,CREATE TABLE [invoice]\n(\n [invoice_id] IN...
6,table,invoice_line,invoice_line,8,CREATE TABLE [invoice_line]\n(\n [invoice_l...
7,table,media_type,media_type,9,CREATE TABLE [media_type]\n(\n [media_type_...
8,table,playlist,playlist,10,CREATE TABLE [playlist]\n(\n [playlist_id] ...
9,table,playlist_track,playlist_track,11,CREATE TABLE [playlist_track]\n(\n [playlis...


The sqlite_master table shows us the names of the tables within the database and the connection points between those tables. Now, let us check the contents of each table:

In [3]:
tables_DF = show_sqlmaster("('table')")
tables = tables_DF["name"]

for name in tables:
    query = "PRAGMA TABLE_INFO("+name+")"
    result = run_query(query)
    display(result)

,cid,name,type,notnull,dflt_value,pk
0,0,album_id,INTEGER,1,None,1
1,1,title,NVARCHAR(160),1,None,0
2,2,artist_id,INTEGER,1,None,0


,cid,name,type,notnull,dflt_value,pk
0,0,artist_id,INTEGER,1,None,1
1,1,name,NVARCHAR(120),0,None,0


,cid,name,type,notnull,dflt_value,pk
0,0,customer_id,INTEGER,1,None,1
1,1,first_name,NVARCHAR(40),1,None,0
2,2,last_name,NVARCHAR(20),1,None,0
3,3,company,NVARCHAR(80),0,None,0
4,4,address,NVARCHAR(70),0,None,0
5,5,city,NVARCHAR(40),0,None,0
6,6,state,NVARCHAR(40),0,None,0
7,7,country,NVARCHAR(40),0,None,0
8,8,postal_code,NVARCHAR(10),0,None,0
9,9,phone,NVARCHAR(24),0,None,0


,cid,name,type,notnull,dflt_value,pk
0,0,employee_id,INTEGER,1,None,1
1,1,last_name,NVARCHAR(20),1,None,0
2,2,first_name,NVARCHAR(20),1,None,0
3,3,title,NVARCHAR(30),0,None,0
4,4,reports_to,INTEGER,0,None,0
5,5,birthdate,DATETIME,0,None,0
6,6,hire_date,DATETIME,0,None,0
7,7,address,NVARCHAR(70),0,None,0
8,8,city,NVARCHAR(40),0,None,0
9,9,state,NVARCHAR(40),0,None,0


,cid,name,type,notnull,dflt_value,pk
0,0,genre_id,INTEGER,1,None,1
1,1,name,NVARCHAR(120),0,None,0


...

,cid,name,type,notnull,dflt_value,pk
0,0,invoice_line_id,INTEGER,1,None,1
1,1,invoice_id,INTEGER,1,None,0
2,2,track_id,INTEGER,1,None,0
3,3,unit_price,"NUMERIC(10,2)",1,None,0
4,4,quantity,INTEGER,1,None,0


,cid,name,type,notnull,dflt_value,pk
0,0,media_type_id,INTEGER,1,None,1
1,1,name,NVARCHAR(120),0,None,0


,cid,name,type,notnull,dflt_value,pk
0,0,playlist_id,INTEGER,1,None,1
1,1,name,NVARCHAR(120),0,None,0


,cid,name,type,notnull,dflt_value,pk
0,0,playlist_id,INTEGER,1,None,1
1,1,track_id,INTEGER,1,None,2


,cid,name,type,notnull,dflt_value,pk
0,0,track_id,INTEGER,1,None,1
1,1,name,NVARCHAR(200),1,None,0
2,2,album_id,INTEGER,0,None,0
3,3,media_type_id,INTEGER,1,None,0
4,4,genre_id,INTEGER,0,None,0
5,5,composer,NVARCHAR(220),0,None,0
6,6,milliseconds,INTEGER,1,None,0
7,7,bytes,INTEGER,0,None,0
8,8,unit_price,"NUMERIC(10,2)",1,None,0


Putting the information above together, we can draw the schema diagram of the database as:
![image.png](./database_structure.png)

### For Whom to Advertise For?

In order to decide which newcomer artists we should advertise for, we need to have an idea about the popularity of the genres of their music in the US market. Here are the artists and genres we are considering:


|Artist Name | Genre |
| --- | --- |
|Regal	| Hip-Hop |
|Red Tone	| Punk |
|Meteor and the Girls|	Pop |
|Slim Jim Bites	| Blues|

In [4]:
query  = ''' 
            WITH sales AS
            ( 
              select sum(il.quantity) total_sales
             
              from invoice i 
              inner join invoice_line il on i.invoice_id = il.invoice_id
              where i.billing_country = 'USA'                  
            )
           
            select 
                g.name genre, 
                sum(il.quantity) tracks_sold,
                round(cast(sum(il.quantity) as float)/cast((select total_sales from sales) as float)*100,1) "tracks_sold(%)"
                
            from invoice i 
            inner join invoice_line il on i.invoice_id = il.invoice_id
            inner join track t on t.track_id = il.track_id 
            inner join genre g on g.genre_id = t.genre_id 
            where i.billing_country = 'USA' 
            
            group by genre
            order by tracks_sold DESC
        '''

run_query(query,index_col = "genre")

,tracks_sold,tracks_sold(%)
genre,,
Rock,561,53.4
Alternative & Punk,130,12.4
Metal,124,11.8
R&B/Soul,53,5.0
Blues,36,3.4
Alternative,35,3.3
Latin,22,2.1
Pop,22,2.1
Hip Hop/Rap,20,1.9


Rock, Punk and Metal seems to be the most popular genres in US for the Chinook  store. 

As for our newcomers, we should advertise for the "Red Tone" group whose music is in the punk genre. Simply because punk is a significant part of our sales (12.4%) whereas blues, pop and hiph-hop makes up a small portion of the sales (3.4%,2.1% and 1.9% consecutively) in USA.

### Performance of the Sales Team

Let us take a look at how the sales team is doing. Our objective is to look for outliers in terms of performance and any trends in the data.

In [5]:
query = '''
            select 
                (e.first_name || " " || e.last_name) rep_name,
                e.title rep_title,
                e.birthdate rep_bd,
                e.hire_date rep_hd,
                e.state rep_state,
                e.country rep_country,
                count(*) num_of_customers,
                sum(i.total) total_purchase
                
            from employee e
            inner join customer c ON c.support_rep_id = e.employee_id
            inner join invoice i ON i.customer_id = c.customer_id
            group by rep_name
            order by total_purchase DESC
            limit 20
'''

run_query(query,index_col = "rep_name")

,rep_title,rep_bd,rep_hd,rep_state,rep_country,num_of_customers,total_purchase
rep_name,,,,,,,
Jane Peacock,Sales Support Agent,1973-08-29 00:00:00,2017-04-01 00:00:00,AB,Canada,212,1731.51
Margaret Park,Sales Support Agent,1947-09-19 00:00:00,2017-05-03 00:00:00,AB,Canada,214,1584.00
Steve Johnson,Sales Support Agent,1965-03-03 00:00:00,2017-10-17 00:00:00,AB,Canada,188,1393.92


Looking at the sales figures, Jane seems to have made the best sales whereas Steve did the worst. Though their sales numbers seem to be in line with the duration these three individuals have been hired.In that sense, there really is no outlier in terms of performance.  

### Market Recommendations

Now, let us take a look at how well the Chinook store is doing in different countries and check which markets has the potential to grow.



In [6]:
query = '''
            WITH 
            sales_per_country AS (
        
                select 
                    billing_country,
                    case
                      when count(distinct customer_id) >1 then billing_country
                      else "Other"
                      end as modified_country,                    
                    count(distinct customer_id) num_of_customers,
                    sum(total) total_sales,
                    avg(total) sales_per_purchase,
                    sum(total)/count(distinct customer_id) sales_per_customer
                
                from invoice
                group by billing_country
            )
                
            select 
                 modified_country country,
                 sum(num_of_customers) customers,
                 sum(total_sales) sales,
                 round(avg(sales_per_purchase),2) per_purchase,
                 round(avg(sales_per_customer),2) per_customer

            from sales_per_country
            group by country            
            order by total_sales DESC 

'''

run_query(query,index_col = "country")


,customers,sales,per_purchase,per_customer
country,,,,
USA,13,1040.49,7.94,80.04
Canada,8,535.59,7.05,66.95
Brazil,5,427.68,7.01,85.54
France,5,389.07,7.78,77.81
Germany,4,334.62,8.16,83.66
Czech Republic,2,273.24,9.11,136.62
United Kingdom,3,245.52,8.77,81.84
Portugal,2,185.13,6.38,92.57
India,2,183.15,8.72,91.58


It is really hard to make a call on this data as the number of customers is extremely small. We also don't have a good grip on what the market's capacity is. It would have helped to have data on youth population as potential customers and GDPs as the buying power of the country. 

Since we don't have any of those, what we can say is to recommend Czech Republic and Portugal as potential growing regions since they have the highest sales per customer. That is to say, if we were able to bring new customers like the ones we have into our system, we can generate significant profits. 

### Single Track vs Whole Album Sales

For this part of the analysis, we are going to entertain a strategy change proposal from the management. In order to reduce the costs, the would like to purchase only the most popular tracks from each album from record companies (instead of purchasing every track from an album).

We need to find out what percentage of the revenue is from whole albums compared to individual tracks to help with this decision.

Couple things to keep in mind:
* Albums with a single track in them (single releases) don't count as albums.
* Users aren't allowed to buy whole albums and add indiviual tracks to a purchase.


In [7]:
query = '''
        WITH
            track_counts AS(
                select 
                    a.album_id,
                    count(distinct t.name) tracks                 
                  
                from track t
                inner join album a on a.album_id = t.album_id 
                group by a.album_id
                order by a.album_id
            ),
                              
            invoice_albums AS(
                select 
                    i.invoice_id,
                    CASE 
                        When count(distinct t.album_id) >1 Then 0
                        Else t.album_id
                        End as album_id,
                    i.total
                from track t
                inner join invoice_line il on il.track_id = t.track_id 
                inner join invoice i on i.invoice_id = il.invoice_id 
                group by i.invoice_id
            ),
            
            album_or_not AS(
            
                select 
                    ia.*,
                    CASE
                      --if there are multiple album_ids
                        When  album_id = 0 Then "Track" 
                      
                      --if the albums are singles
                        When  (                         
                                select tc.tracks from track_counts tc
                                where tc.album_id = ia.album_id 
                              ) =1 Then "Track"
                     
                      --if tracks from invoices matches all of album tracks
                        When (                          
                            select t.track_id from track t
                            where t.album_id = ia.album_id

                            EXCEPT

                            select il.track_id from invoice_line il
                            where il.invoice_id = ia.invoice_id) is Null Then "Album"
                       
                       --Anything else
                        Else "Track"
                    End as sale_type

                from invoice_albums ia
                order by invoice_id
            ),            
             
            sorted AS(
                select 
                    sale_type,
                    count(*) transactions, 
                    sum(total) sales
                from album_or_not aon
                group by sale_type
            )
            
            select 
                sale_type,
                transactions,
                round(transactions/cast((select count(*) from album_or_not) as float)*100,1)  "transactions %",
                sales,
                round(sales/(select sum(total) from album_or_not)*100,1) "sales %"
             from sorted
        
'''

run_query(query)

,sale_type,transactions,transactions %,sales,sales %
0,Album,111,18.1,1458.27,31.0
1,Track,503,81.9,3251.16,69.0


The album sales seems to be a significant piece of the overall revenue (31%) and it doesn't seem to be a good idea to stop buying albums. Yet, we need to have an idea about how much money can be saved by not buying the albums.If the savings can offset this loss, then the decision might have a chance.